# <span style = "color :  #ca6f1e ">Activité Python : Dynamique des populations - Modèle de Verhulst, Partie 2 </span>
## Apparition des $2^n$-cycles

On continue notre exploration dans cette deuxième partie du modèle de Verhulst !

Cet outil va te permettre d'exécuter du code Python facilement, à travers ce que j'ai déjà codé. 
 
Tu verras que beaucoup de commandes et lignes de codes sont très avancées. Mais ne t'inquiète pas, l'objectif n'est pas que tu comprennes tout le code mais plutôt que tu découvres la puissance de l'informatique pour ce genre de calculs et pour l'intuition qu'elle peut t'apporter, mais aussi ses limites. Tu auras donc à ta disposition un outil pour modifier les paramètres des simulations et essayer de comprendre l'influence des paramètres sur l'évolution. Bonne découverte !

<b>Remarque</b> : au début, appuie sur Cell->Run All pour tout lancer. Pour activer une cellule de code particulière (comme après une modification), clique dessus et appuie sur Cells->Run Cells, ou Shift+Enter ;)

In [7]:
import os
import sys
import random as rd
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import ColumnDataSource, output_file, show

output_notebook(hide_banner=True)

#fonction f_µ
def f_mu(mu):
    return lambda x:mu*x*(1-x)

### $\underline{\text{De la convergence vers un point fixe à l'émergence des }2^n \text{-cycles}}$

Tu as pu voir qu'au passage de 3, la suite de la population tend à osciller entre deux valeurs. Mais que se passe-t-il lorsque $\mu$ va encore plus loin ?

Je te propose donc de faire varier $\mu$ sur une plage plus grande que précédemment pour voir les répercussions occasionnées. Tu peux faire varier l'élément initial de la suite et le nombre d'itérations considérées : seules les 10 dernières sont présentées sur les deux graphiques.

Celui de gauche permet d'observer $u_{n+1}$ en fonction de $u_n$ (on voit graphiquement les n-cycles, par le nombre de sommets de la figure) en reliant ($u_n$,$u_{n+1}$) à ($u_{n+1}$,$u_{n+2}$), et celui de droite correspond au tracé des 17 dernières valeurs de la suite, pour observer simplement le comportement (attention pour $\mu \leq 3$ : la courbe de droite n'a pas forcément l'air horizontale, mais l'axe des ordonnées montre que les variations sont très faibles !).

In [8]:
liste_couleur=[(255,0,0),(255,16,0),(255,32,0),(255,48,0),(255,64,0),(255,80,0),(255,96,0),(255,112,0),(255,128,0),(255,144,0),(255,160,0),(255,176,0),(255,192,0),(255,208,0),(255,224,0),(255,240,0),(255,255,0)]

#fonction d'affichage des valeurs
def affichage_points(liste, fig):
    for i in range(len(liste)-2):
        fig.line(liste[i:i+2],liste[i+1:i+3], color=liste_couleur[i])

#fonction d'affichage des itérations
def affichage_evolution(x0, mu, nb_itérations):
    f=f_mu(mu)
    liste_x=[]
    x=x0
    for i in range(0,nb_itérations):
        x=f(x)
        if i>=nb_itérations-17:
            liste_x.append(x)
    fig = figure(width=470, height=400, title="Tracé du point suivant en fonction du précédent")
    affichage_points(liste_x, fig)     
    
    fig1 = figure(width=470, height=400, title="Tracé des dernières valeurs de la suite")
    fig1.line(list(range(nb_itérations-16, nb_itérations+1)), liste_x)
    
    show(row(fig, fig1))

In [9]:
interact(affichage_evolution, x0=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1, continuous_update=False), mu=widgets.FloatSlider(min=2.8, max=3.58, step=0.01, value=2.8, continuous_update=False), nb_itérations=widgets.IntSlider(min=10, max=10000, step=10, value=20, continuous_update=False))

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='x0', max=1.0), FloatSlider(…

<function __main__.affichage_evolution(x0, mu, nb_itérations)>

On observe différents comportements et variations lors de la variation de $\mu$ :
- la $\textbf{convergence vers une valeur}$ correspond à une absence de tracé à gauche (ou à une longueur très faible) et à un trait horizontal à droite (ou de faibles variations autour de la limite). C'est bien ce que l'on observe pour $\mu\leq 3$ (elle est très lente pour $\mu =3$ mais tout de même présente) !
- on va observer $\textbf{l'apparition d'un 2-cycle}$ pour $\mu >3$. Dans ce cas, on n'observe plus un point mais bien un trait entre deux valeurs bien distinctes à gauche, et une alternance régulière entre deux valeurs à droite !
- un autre changement a lieu entre $\mu=3,44$ et $\mu=3,45$ : avec un nombre suffisant d'itérations, c'est bien un $\textbf{4-cycle qui apparaît}$ !
- ce phénomène se répète entre $\mu=3,54$ et $\mu=3,55$ : on passe alors $\textbf{d'un 4-cycle à un 8-cycle}$ !
- on arrive même à un 16-cycle pour $\mu=3,57$ !

Tu as donc pu observer de toi-même le comportement de la suite, et voir l'apparition des $2^n$-cycles ! Mais au lieu de s'arrêter jusqu'à 16, allons un peu plus loin !

### $\underline{\text{La cascade de doublement de périodes}}$

Pour visualiser ce comportement, on considère différentes valeurs de $\mu$ pour lesquelles on effectue 10 000 itérations à partir de la valeur initiale 0,1 (elle a été choisie arbitrairement pour éviter de tomber sur un point fixe). On considère alors le dernier terme de la suite obtenue, noté $u_{N_{fin}}$ et l'on revient en arrière jusqu'à obtenir le premier terme suffisament proche de $u_{N_{fin}}$, noté $u_{N_{précédent}}$ (la précision a été prise de manière arbitraire à $10^{-8}$, pour être sûr de ne pas considérer deux membres distincts d'un cycle plus grand).

On peut donc considérer que la suite a convergé vers un cycle de longueur $N_{fin}-N_{précédent}$ et que les éléments $u_p$ pour $p \in \{N_{précédent}+1,\dots,N_{fin}\}$ sont les éléments de ce cycle.

Les délimitations des différents $\mu_n$ tels que le cycle soit de longueur $2^n$ entre $\mu_n$ et $\mu_{n+1}$ ont été obtenues à partir d'une estimation asymptotique, qui donne dès $n=3$ une très bonne approximation. 

In [10]:
#récupération de N_fin-N_précédent
def min_cycle_fin(liste_verif, liste_iter, epsi):
    for k in liste_verif:
        if abs(liste_iter[-1]-liste_iter[-1-k])<epsi:
            return k
    return 0

#calcul des approximations des éléments des cycles pour chaque µ
def liste_cycle_fin(mu, x0, epsi):
    f=f_mu(mu)
    liste_iter=[x0]
    liste_verif=[2**j for j in range(0,int(np.log(10000)/np.log(2)))]
    for k in range(1,10000):
        liste_iter.append(f(liste_iter[-1]))
    c=min_cycle_fin(liste_verif, liste_iter,epsi)
    return np.array(liste_iter[len(liste_iter)-int(c):])

#découpe chaque intervalle ]mu_(n), mu_(n+1)[ approximé en nb valeurs, jusqu'à n
def decoupe_intervalle(n,nb): 
    m1=3
    m2=1+np.sqrt(6)
    m3=3.544090
    m4=3.564407
    d=4.66921166091029906
    liste=np.concatenate((np.linspace(0,3,1000), np.linspace(m1,m2,nb), np.linspace(m2,m3,nb), np.linspace(m3,m4,nb)))
    a=m3
    b=m4
    for i in range(5,n):
        m=b+(b-a)/d
        liste=np.concatenate((liste, np.linspace(b,m,nb)))
        a=b
        b=m
    return liste

In [11]:
#affichage de la cascade de doublement
def cascade_doublement(nb_cycles,nb_points_par_cycle):
    N_max=nb_cycles
    nb=nb_points_par_cycle
    liste_x=[]
    liste_y=[]
    liste_long_cycle=[]
    print("On note :")
    print("- nb_cycles le nombre de 2^n-cycles que l'on veut afficher")
    print("- nb_points_par_cycle le nombre de points pour chaque 2^n-cycle")
    print("N'hésite pas à utiliser l'outil 'Zoom', le deuxième à droite en partant du haut, pour sélectionner une zone à observer plus précisément !")
    for mu in decoupe_intervalle(N_max,nb):
        cycle=liste_cycle_fin(mu,0.1,0.00000001)
        liste_long_cycle.append(cycle.shape[0])
        for k in range(cycle.shape[0]):
            liste_x.append(mu)
            liste_y.append(cycle[k])
    fig = figure(width=950, height=400, title="Cascade de doublement de périodes")
    fig.line(np.linspace(1/3000,3,3000),[max(1-1/z,0) for z in np.linspace(1/3000,3,3000)], legend_label="Point fixe attractif", color="black")
    fig.x(liste_x, liste_y, legend_label="Limites")
    fig.legend.location = "top_left"       
    show(fig)

In [13]:
interact(cascade_doublement, nb_cycles=widgets.IntSlider(min=1, max=15, step=1, value=3, continuous_update=False), nb_points_par_cycle=widgets.IntSlider(min=10, max=100, step=10, value=30, continuous_update=False))

interactive(children=(IntSlider(value=3, continuous_update=False, description='nb_cycles', max=15, min=1), Int…

<function __main__.cascade_doublement(nb_cycles, nb_points_par_cycle)>

Pour $\mu \leq 3$, on retrouve bien la convergence vers l'unique point fixe attractif : 0 jusqu'à $\mu=1$, et $1-\frac{1}{\mu}$ ensuite.

Ensuite, à partir de $\mu=3$, on observe bien l'apparition des 2-cycles, qui se fait sous la forme d'une bifurcation. Ce phénomène de séparation va se répéter au niveau de ces deux valeurs lorsque $\mu$ atteint une nouvelle valeur (on peut prouver qu'il s'agit de $1+\sqrt{6}$) : le 2-cycle se sépare alors en un 4-cycle... Ce phénomène se répète alors systématiquement jusqu'à une certaine valeur $\mu_{\infty}$ : c'est ce que l'on appelle $\textbf{la cascade de doublement de périodes !}$

$\color{red}{\text{Pour mieux observer ce phénomène, n'hésite pas à utiliser l'outil "Zoom", le deuxième à droite en partant du haut, pour sélectionner une zone à observer plus précisément !}}$